In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('all-data.csv', names=['Sentiment', 'Headline'], encoding='latin1')

In [3]:
df

,Sentiment,Headline
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [5]:
X = df['Headline']
y = df['Sentiment']

In [6]:
# Encode labels
encoder = LabelEncoder()
y_enc = encoder.fit_transform(y)

In [7]:
# Tokenize
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=30)

In [11]:
import pickle
# Save tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
# Save encoder
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)
# Save model
model.save('sentiment_model.h5')

D:\Anaconda\envs\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_enc, test_size=0.2, random_state=42)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
cw_dict = {i: w for i, w in enumerate(class_weights)}

In [15]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=30),
    LSTM(64),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:
model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    class_weight=cw_dict
)

Epoch 1/10
109/109 [==============================] - 5s 21ms/step - loss: 1.0199 - accuracy: 0.4842 - val_loss: 0.8304 - val_accuracy: 0.6675
Epoch 2/10
109/109 [==============================] - 2s 15ms/step - loss: 0.6476 - accuracy: 0.7351 - val_loss: 0.7407 - val_accuracy: 0.6598
Epoch 3/10
109/109 [==============================] - 1s 13ms/step - loss: 0.3333 - accuracy: 0.8713 - val_loss: 0.7808 - val_accuracy: 0.6881
Epoch 4/10
109/109 [==============================] - 2s 17ms/step - loss: 0.2104 - accuracy: 0.9321 - val_loss: 0.9092 - val_accuracy: 0.6959
Epoch 5/10
109/109 [==============================] - 2s 19ms/step - loss: 0.1242 - accuracy: 0.9601 - val_loss: 1.2134 - val_accuracy: 0.6804
Epoch 6/10
109/109 [==============================] - 2s 16ms/step - loss: 0.1005 - accuracy: 0.9676 - val_loss: 1.2791 - val_accuracy: 0.6985
Epoch 7/10
109/109 [==============================] - 2s 17ms/step - loss: 0.0874 - accuracy: 0.9710 - val_loss: 1.2223 - val_accuracy: 0.6907

In [18]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

31/31 [==============================] - 0s 8ms/step - loss: 1.1791 - accuracy: 0.7639
Test Accuracy: 76.39%
